In [1]:
import numpy as np
import pandas as pd
import re
from os.path import join, isfile
from tqdm import tqdm
from sklearn import metrics

In [19]:
max_features=100000
maxlen=150
embed_size=300

file_embedding_txt = join('E://DM//NLP//WordVec', 'glove.840B.300d.txt')
path_tmp = join('E:\\DM\\NLP\\TMP_MEMORY','ToxicCommentClassification')
file_model = join(path_tmp, 'weights_base.best.hdf5')
file_embedding_matrix = join(path_tmp, 'embedding_matrix.hkl')
file_train = join(path_tmp, 'train_clean.csv')
file_test = join(path_tmp, 'test_clean.csv')
file_submission = join(path_tmp, 'submission.csv')
file_submission_example = join(path_tmp, 'data', 'sample_submission.csv')

In [3]:
labels = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

def GetData(file_train, file_test):
    train = pd.read_csv(file_train, encoding='utf-8')
    test = pd.read_csv(file_test, encoding='utf-8')
    train["comment_text"].fillna('fillna')
    test["comment_text"].fillna('fillna')
    X_train = train["comment_text"].apply(str)
    X_test = test["comment_text"].apply(str)

    y_train = train[labels]
    merge = pd.concat([X_train, X_test]).reset_index(drop=True)
    merge = merge.astype('str')
    return X_train, X_test, y_train, merge


X_train, X_test, y_train, merge = GetData(file_train, file_test)




In [4]:
X_train.describe()

count     159571
unique    157881
top          nan
freq          52
Name: comment_text, dtype: object

In [5]:
from keras.preprocessing import text, sequence

tokenizer = text.Tokenizer(num_words=max_features, lower=True)
copus = list(X_train) + list(X_test)
copus = [str(x) for x in copus]
tokenizer.fit_on_texts(copus)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)
X_train_pad = sequence.pad_sequences(X_train_seq, maxlen=maxlen)
X_test_pad = sequence.pad_sequences(X_test_seq, maxlen=maxlen)

length_seq = [len(document) for document in X_train_seq[:10]]
print('length of X_train_seq:' + str(length_seq))
print('shape of X_train_pad: ' + str(X_train_pad.shape))

D:\Program Files\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


length of X_train_seq:[26, 12, 22, 57, 5, 6, 4, 6, 45, 4]
shape of X_train_pad: (159571, 150)


In [6]:
def ReadWord2Vec():
    embeddings_index = {}
    with open(file_embedding_txt, encoding='utf8') as f:
        for line in tqdm(f):
            values = line.rstrip().rsplit(' ')
            assert len(values)>=300
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    return embeddings_index

def GetEmbeddingStat(embeddings_index):
    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    return emb_mean, emb_std

def CreateEmbeddingMatrix():
    embeddings_index = ReadWord2Vec()
    emb_mean, emb_std = GetEmbeddingStat(embeddings_index)
    word_index = tokenizer.word_index
    num_words = min(max_features, len(word_index) + 1)
    embedding_matrix = np.random.normal(emb_mean, emb_std, (num_words, embed_size))
    print(len(embeddings_index))
    print(len(word_index))
    print(max_features)
    print(embedding_matrix.shape)
    
    num_missed = 0
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
        else:
            num_missed +=1
    print('number of words that are not found in the embedding_vector: ' + str(num_missed))
    return embedding_matrix


    


### How to store Large numpy arrays on disk in python
* stackoverflow question: https://stackoverflow.com/questions/9619199/best-way-to-preserve-numpy-arrays-on-disk
* pickle VS. hdf5: https://shocksolution.com/2010/01/10/storing-large-numpy-arrays-on-disk-python-pickle-vs-hdf5adsf/
* hickle github: https://github.com/telegraphic/hickle
* HDF is good for storing large numpy arrays
* hickle aims to be exactly the same in usage as pickle, but using HDF for data storage

```python
import os
import hickle as hkl
import numpy as np
    
# Create a numpy array of data
array_obj = np.ones(32768, dtype='float32')
    
# Dump to file
hkl.dump(array_obj, 'test.hkl', mode='w')
    
# Dump data, with compression
hkl.dump(array_obj, 'test_gzip.hkl', mode='w', compression='gzip')
  
# Compare filesizes
print('uncompressed: %i bytes' % os.path.getsize('test.hkl'))
print('compressed:   %i bytes' % os.path.getsize('test_gzip.hkl'))
    
# Load data
array_hkl = hkl.load('test_gzip.hkl')
    
# Check the two are the same file
assert array_hkl.dtype == array_obj.dtype
assert np.all((array_hkl, array_obj))
```


In [7]:
import hickle as hkl
def GetEmbeddingMatrix():
    if isfile(file_embedding_matrix):
        embedding_matrix = hkl.load(file_embedding_matrix)
        hkl.dump(embedding_matrix, file_embedding_matrix, mode='w')
    else:
        embedding_matrix = CreateEmbeddingMatrix()
    
    return embedding_matrix

embedding_matrix = GetEmbeddingMatrix()


2196018it [05:34, 6572.99it/s]


2196017
291655
100000
(100000, 300)
number of words that are not found in the embedding_vector: 21248


In [8]:
from keras.callbacks import Callback
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: {:d} - score: {:.6f}".format(epoch+1, score))

In [9]:
# A blog about LSTM-CNNs:
# http://konukoii.com/blog/2018/02/19/twitter-sentiment-analysis-using-combined-lstm-cnn-models/
from keras.layers import Dense,Input,Bidirectional,Activation,Conv1D,GRU, Dropout,Embedding
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras.models import Model
from keras.optimizers import Adam

def GetModel():
    sequence_input = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix],trainable = False)(sequence_input)
    x = SpatialDropout1D(0.2)(x)
    x = Bidirectional(GRU(128, return_sequences=True,dropout=0.1,recurrent_dropout=0.1))(x)
    x = Conv1D(64, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    x = concatenate([avg_pool, max_pool]) 
    # x = Dense(128, activation='relu')(x)
    # x = Dropout(0.1)(x)
    preds = Dense(6, activation="sigmoid")(x)
    model = Model(sequence_input, preds)
    model.compile(loss='binary_crossentropy',optimizer=Adam(lr=1e-3),metrics=['accuracy'])
    return model

model = GetModel()
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 150)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 150, 300)     30000000    input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_1 (SpatialDro (None, 150, 300)     0           embedding_1[0][0]                
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 150, 256)     329472      spatial_dropout1d_1[0][0]        
__________________________________________________________________________________________________
conv1d_1 (

In [12]:
from sklearn.model_selection import train_test_split
X_tra, X_val, y_tra, y_val = train_test_split(X_train_pad, y_train, train_size=0.9, random_state=233)

D:\Program Files\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [13]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
checkpoint = ModelCheckpoint(file_model, monitor='val_acc', save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5)
ra_val = RocAucEvaluation(validation_data=(X_val, y_val), interval = 1)
callbacks_list = [ra_val,checkpoint, early]

In [14]:
for l in model.layers:
    print(l.name)
for l in model.layers[:1]:
    l.trainable = False
epochs = 1
batch_size = 128
model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val),callbacks = callbacks_list,verbose=1)    


Train on 143613 samples, validate on 15958 samples
Epoch 1/1


 25856/143613 [====>.........................] - ETA: 1:29:20 - loss: 0.6054 - acc: 0.66 - ETA: 51:43 - loss: 0.4402 - acc: 0.8223 - ETA: 39:34 - loss: 0.3563 - acc: 0.86 - ETA: 33:11 - loss: 0.3184 - acc: 0.88 - ETA: 29:39 - loss: 0.2914 - acc: 0.90 - ETA: 27:28 - loss: 0.2588 - acc: 0.91 - ETA: 25:26 - loss: 0.2379 - acc: 0.92 - ETA: 24:13 - loss: 0.2262 - acc: 0.93 - ETA: 23:25 - loss: 0.2162 - acc: 0.93 - ETA: 22:31 - loss: 0.2059 - acc: 0.93 - ETA: 21:52 - loss: 0.2037 - acc: 0.94 - ETA: 21:24 - loss: 0.2018 - acc: 0.94 - ETA: 20:50 - loss: 0.2001 - acc: 0.94 - ETA: 20:22 - loss: 0.1978 - acc: 0.94 - ETA: 19:54 - loss: 0.1930 - acc: 0.94 - ETA: 19:34 - loss: 0.1919 - acc: 0.94 - ETA: 19:19 - loss: 0.1912 - acc: 0.94 - ETA: 18:57 - loss: 0.1897 - acc: 0.94 - ETA: 18:44 - loss: 0.1847 - acc: 0.94 - ETA: 18:25 - loss: 0.1823 - acc: 0.95 - ETA: 18:16 - loss: 0.1806 - acc: 0.95 - ETA: 18:04 - loss: 0.1756 - acc: 0.95 - ETA: 17:50 - loss: 0.1727 - acc: 0.95 - ETA: 17:40 - loss: 0.1714 -

 51968/143613 [=========>....................] - ETA: 13:19 - loss: 0.0849 - acc: 0.97 - ETA: 13:17 - loss: 0.0847 - acc: 0.97 - ETA: 13:16 - loss: 0.0845 - acc: 0.97 - ETA: 13:15 - loss: 0.0845 - acc: 0.97 - ETA: 13:13 - loss: 0.0843 - acc: 0.97 - ETA: 13:13 - loss: 0.0843 - acc: 0.97 - ETA: 13:12 - loss: 0.0843 - acc: 0.97 - ETA: 13:11 - loss: 0.0841 - acc: 0.97 - ETA: 13:10 - loss: 0.0840 - acc: 0.97 - ETA: 13:09 - loss: 0.0839 - acc: 0.97 - ETA: 13:09 - loss: 0.0838 - acc: 0.97 - ETA: 13:08 - loss: 0.0837 - acc: 0.97 - ETA: 13:08 - loss: 0.0837 - acc: 0.97 - ETA: 13:07 - loss: 0.0838 - acc: 0.97 - ETA: 13:06 - loss: 0.0837 - acc: 0.97 - ETA: 13:05 - loss: 0.0836 - acc: 0.97 - ETA: 13:04 - loss: 0.0836 - acc: 0.97 - ETA: 13:03 - loss: 0.0834 - acc: 0.97 - ETA: 13:02 - loss: 0.0833 - acc: 0.97 - ETA: 13:01 - loss: 0.0831 - acc: 0.97 - ETA: 12:59 - loss: 0.0829 - acc: 0.97 - ETA: 12:58 - loss: 0.0828 - acc: 0.97 - ETA: 12:58 - loss: 0.0826 - acc: 0.97 - ETA: 12:57 - loss: 0.0826 - acc

 78080/143613 [===============>..............] - ETA: 10:21 - loss: 0.0689 - acc: 0.97 - ETA: 10:20 - loss: 0.0689 - acc: 0.97 - ETA: 10:19 - loss: 0.0688 - acc: 0.97 - ETA: 10:18 - loss: 0.0688 - acc: 0.97 - ETA: 10:17 - loss: 0.0687 - acc: 0.97 - ETA: 10:16 - loss: 0.0687 - acc: 0.97 - ETA: 10:15 - loss: 0.0686 - acc: 0.97 - ETA: 10:14 - loss: 0.0686 - acc: 0.97 - ETA: 10:13 - loss: 0.0687 - acc: 0.97 - ETA: 10:12 - loss: 0.0687 - acc: 0.97 - ETA: 10:12 - loss: 0.0686 - acc: 0.97 - ETA: 10:11 - loss: 0.0686 - acc: 0.97 - ETA: 10:10 - loss: 0.0685 - acc: 0.97 - ETA: 10:09 - loss: 0.0684 - acc: 0.97 - ETA: 10:08 - loss: 0.0684 - acc: 0.97 - ETA: 10:07 - loss: 0.0684 - acc: 0.97 - ETA: 10:05 - loss: 0.0683 - acc: 0.97 - ETA: 10:05 - loss: 0.0684 - acc: 0.97 - ETA: 10:04 - loss: 0.0683 - acc: 0.97 - ETA: 10:03 - loss: 0.0682 - acc: 0.97 - ETA: 10:02 - loss: 0.0682 - acc: 0.97 - ETA: 10:01 - loss: 0.0682 - acc: 0.97 - ETA: 10:00 - loss: 0.0683 - acc: 0.97 - ETA: 9:59 - loss: 0.0683 - acc:

104192/143613 [====================>.........] - ETA: 7:24 - loss: 0.0633 - acc: 0.978 - ETA: 7:23 - loss: 0.0632 - acc: 0.978 - ETA: 7:22 - loss: 0.0632 - acc: 0.978 - ETA: 7:22 - loss: 0.0632 - acc: 0.978 - ETA: 7:21 - loss: 0.0632 - acc: 0.978 - ETA: 7:20 - loss: 0.0632 - acc: 0.978 - ETA: 7:19 - loss: 0.0632 - acc: 0.978 - ETA: 7:18 - loss: 0.0632 - acc: 0.978 - ETA: 7:17 - loss: 0.0632 - acc: 0.978 - ETA: 7:16 - loss: 0.0632 - acc: 0.978 - ETA: 7:15 - loss: 0.0632 - acc: 0.978 - ETA: 7:15 - loss: 0.0632 - acc: 0.978 - ETA: 7:14 - loss: 0.0632 - acc: 0.978 - ETA: 7:13 - loss: 0.0631 - acc: 0.978 - ETA: 7:12 - loss: 0.0631 - acc: 0.978 - ETA: 7:11 - loss: 0.0630 - acc: 0.978 - ETA: 7:10 - loss: 0.0630 - acc: 0.978 - ETA: 7:09 - loss: 0.0630 - acc: 0.978 - ETA: 7:08 - loss: 0.0630 - acc: 0.978 - ETA: 7:08 - loss: 0.0630 - acc: 0.978 - ETA: 7:07 - loss: 0.0629 - acc: 0.978 - ETA: 7:06 - loss: 0.0629 - acc: 0.978 - ETA: 7:05 - loss: 0.0629 - acc: 0.978 - ETA: 7:04 - loss: 0.0629 - acc:

130304/143613 [==========================>...] - ETA: 4:27 - loss: 0.0602 - acc: 0.978 - ETA: 4:27 - loss: 0.0602 - acc: 0.978 - ETA: 4:26 - loss: 0.0602 - acc: 0.978 - ETA: 4:25 - loss: 0.0602 - acc: 0.978 - ETA: 4:24 - loss: 0.0602 - acc: 0.978 - ETA: 4:23 - loss: 0.0602 - acc: 0.978 - ETA: 4:22 - loss: 0.0602 - acc: 0.978 - ETA: 4:21 - loss: 0.0601 - acc: 0.978 - ETA: 4:20 - loss: 0.0601 - acc: 0.978 - ETA: 4:20 - loss: 0.0601 - acc: 0.978 - ETA: 4:19 - loss: 0.0601 - acc: 0.978 - ETA: 4:18 - loss: 0.0601 - acc: 0.979 - ETA: 4:17 - loss: 0.0601 - acc: 0.979 - ETA: 4:16 - loss: 0.0601 - acc: 0.979 - ETA: 4:15 - loss: 0.0601 - acc: 0.979 - ETA: 4:14 - loss: 0.0600 - acc: 0.979 - ETA: 4:13 - loss: 0.0600 - acc: 0.979 - ETA: 4:13 - loss: 0.0600 - acc: 0.979 - ETA: 4:12 - loss: 0.0599 - acc: 0.979 - ETA: 4:11 - loss: 0.0599 - acc: 0.979 - ETA: 4:10 - loss: 0.0600 - acc: 0.979 - ETA: 4:09 - loss: 0.0600 - acc: 0.979 - ETA: 4:08 - loss: 0.0599 - acc: 0.979 - ETA: 4:07 - loss: 0.0599 - acc:

143488/143613 [============================>.] - ETA: 1:29 - loss: 0.0578 - acc: 0.979 - ETA: 1:28 - loss: 0.0578 - acc: 0.979 - ETA: 1:27 - loss: 0.0578 - acc: 0.979 - ETA: 1:26 - loss: 0.0578 - acc: 0.979 - ETA: 1:25 - loss: 0.0578 - acc: 0.979 - ETA: 1:24 - loss: 0.0578 - acc: 0.979 - ETA: 1:24 - loss: 0.0578 - acc: 0.979 - ETA: 1:23 - loss: 0.0578 - acc: 0.979 - ETA: 1:22 - loss: 0.0578 - acc: 0.979 - ETA: 1:21 - loss: 0.0578 - acc: 0.979 - ETA: 1:20 - loss: 0.0578 - acc: 0.979 - ETA: 1:19 - loss: 0.0578 - acc: 0.979 - ETA: 1:18 - loss: 0.0577 - acc: 0.979 - ETA: 1:18 - loss: 0.0578 - acc: 0.979 - ETA: 1:17 - loss: 0.0577 - acc: 0.979 - ETA: 1:16 - loss: 0.0577 - acc: 0.979 - ETA: 1:15 - loss: 0.0577 - acc: 0.979 - ETA: 1:14 - loss: 0.0577 - acc: 0.979 - ETA: 1:13 - loss: 0.0577 - acc: 0.979 - ETA: 1:12 - loss: 0.0576 - acc: 0.979 - ETA: 1:12 - loss: 0.0576 - acc: 0.979 - ETA: 1:11 - loss: 0.0576 - acc: 0.979 - ETA: 1:10 - loss: 0.0576 - acc: 0.979 - ETA: 1:09 - loss: 0.0576 - acc:

In [18]:
epochs = 2
batch_size = 128
for l in model.layers:
    l.trainable = True
model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val),callbacks = callbacks_list,verbose=1)   


Train on 143613 samples, validate on 15958 samples
Epoch 1/2


 25984/143613 [====>.........................] - ETA: 14:06 - loss: 0.0465 - acc: 0.98 - ETA: 14:01 - loss: 0.0410 - acc: 0.98 - ETA: 14:41 - loss: 0.0450 - acc: 0.98 - ETA: 14:32 - loss: 0.0488 - acc: 0.98 - ETA: 14:29 - loss: 0.0487 - acc: 0.98 - ETA: 14:15 - loss: 0.0479 - acc: 0.98 - ETA: 14:19 - loss: 0.0516 - acc: 0.98 - ETA: 14:19 - loss: 0.0485 - acc: 0.98 - ETA: 14:21 - loss: 0.0502 - acc: 0.98 - ETA: 14:21 - loss: 0.0503 - acc: 0.98 - ETA: 14:38 - loss: 0.0492 - acc: 0.98 - ETA: 14:39 - loss: 0.0480 - acc: 0.98 - ETA: 14:57 - loss: 0.0470 - acc: 0.98 - ETA: 14:53 - loss: 0.0468 - acc: 0.98 - ETA: 14:47 - loss: 0.0461 - acc: 0.98 - ETA: 14:43 - loss: 0.0468 - acc: 0.98 - ETA: 14:40 - loss: 0.0476 - acc: 0.98 - ETA: 14:51 - loss: 0.0463 - acc: 0.98 - ETA: 14:45 - loss: 0.0455 - acc: 0.98 - ETA: 14:51 - loss: 0.0450 - acc: 0.98 - ETA: 14:50 - loss: 0.0442 - acc: 0.98 - ETA: 14:46 - loss: 0.0454 - acc: 0.98 - ETA: 14:52 - loss: 0.0462 - acc: 0.98 - ETA: 14:49 - loss: 0.0459 - acc

 52096/143613 [=========>....................] - ETA: 12:08 - loss: 0.0459 - acc: 0.98 - ETA: 12:07 - loss: 0.0459 - acc: 0.98 - ETA: 12:06 - loss: 0.0458 - acc: 0.98 - ETA: 12:06 - loss: 0.0458 - acc: 0.98 - ETA: 12:05 - loss: 0.0458 - acc: 0.98 - ETA: 12:04 - loss: 0.0458 - acc: 0.98 - ETA: 12:03 - loss: 0.0459 - acc: 0.98 - ETA: 12:04 - loss: 0.0459 - acc: 0.98 - ETA: 12:03 - loss: 0.0460 - acc: 0.98 - ETA: 12:03 - loss: 0.0459 - acc: 0.98 - ETA: 12:02 - loss: 0.0460 - acc: 0.98 - ETA: 12:01 - loss: 0.0459 - acc: 0.98 - ETA: 12:00 - loss: 0.0459 - acc: 0.98 - ETA: 12:01 - loss: 0.0459 - acc: 0.98 - ETA: 12:00 - loss: 0.0459 - acc: 0.98 - ETA: 11:59 - loss: 0.0460 - acc: 0.98 - ETA: 11:59 - loss: 0.0460 - acc: 0.98 - ETA: 11:58 - loss: 0.0460 - acc: 0.98 - ETA: 11:58 - loss: 0.0459 - acc: 0.98 - ETA: 11:58 - loss: 0.0459 - acc: 0.98 - ETA: 11:58 - loss: 0.0460 - acc: 0.98 - ETA: 11:58 - loss: 0.0460 - acc: 0.98 - ETA: 11:59 - loss: 0.0459 - acc: 0.98 - ETA: 11:58 - loss: 0.0460 - acc

 78208/143613 [===============>..............] - ETA: 9:55 - loss: 0.0457 - acc: 0.982 - ETA: 9:54 - loss: 0.0457 - acc: 0.982 - ETA: 9:53 - loss: 0.0456 - acc: 0.982 - ETA: 9:52 - loss: 0.0456 - acc: 0.982 - ETA: 9:51 - loss: 0.0456 - acc: 0.982 - ETA: 9:50 - loss: 0.0456 - acc: 0.982 - ETA: 9:50 - loss: 0.0456 - acc: 0.982 - ETA: 9:49 - loss: 0.0456 - acc: 0.982 - ETA: 9:48 - loss: 0.0456 - acc: 0.982 - ETA: 9:47 - loss: 0.0456 - acc: 0.982 - ETA: 9:46 - loss: 0.0456 - acc: 0.982 - ETA: 9:45 - loss: 0.0455 - acc: 0.982 - ETA: 9:44 - loss: 0.0456 - acc: 0.982 - ETA: 9:43 - loss: 0.0455 - acc: 0.982 - ETA: 9:42 - loss: 0.0455 - acc: 0.982 - ETA: 9:41 - loss: 0.0455 - acc: 0.982 - ETA: 9:40 - loss: 0.0455 - acc: 0.982 - ETA: 9:39 - loss: 0.0454 - acc: 0.982 - ETA: 9:38 - loss: 0.0455 - acc: 0.982 - ETA: 9:37 - loss: 0.0455 - acc: 0.982 - ETA: 9:36 - loss: 0.0455 - acc: 0.982 - ETA: 9:35 - loss: 0.0455 - acc: 0.982 - ETA: 9:34 - loss: 0.0455 - acc: 0.982 - ETA: 9:33 - loss: 0.0455 - acc:

104320/143613 [====================>.........] - ETA: 6:54 - loss: 0.0457 - acc: 0.982 - ETA: 6:53 - loss: 0.0456 - acc: 0.982 - ETA: 6:52 - loss: 0.0456 - acc: 0.982 - ETA: 6:51 - loss: 0.0456 - acc: 0.982 - ETA: 6:50 - loss: 0.0456 - acc: 0.982 - ETA: 6:49 - loss: 0.0456 - acc: 0.982 - ETA: 6:48 - loss: 0.0456 - acc: 0.982 - ETA: 6:48 - loss: 0.0456 - acc: 0.982 - ETA: 6:47 - loss: 0.0455 - acc: 0.982 - ETA: 6:46 - loss: 0.0455 - acc: 0.982 - ETA: 6:45 - loss: 0.0455 - acc: 0.982 - ETA: 6:44 - loss: 0.0456 - acc: 0.982 - ETA: 6:43 - loss: 0.0455 - acc: 0.982 - ETA: 6:42 - loss: 0.0456 - acc: 0.982 - ETA: 6:42 - loss: 0.0456 - acc: 0.982 - ETA: 6:41 - loss: 0.0455 - acc: 0.982 - ETA: 6:40 - loss: 0.0455 - acc: 0.982 - ETA: 6:39 - loss: 0.0455 - acc: 0.982 - ETA: 6:38 - loss: 0.0455 - acc: 0.982 - ETA: 6:37 - loss: 0.0455 - acc: 0.982 - ETA: 6:37 - loss: 0.0455 - acc: 0.982 - ETA: 6:36 - loss: 0.0455 - acc: 0.982 - ETA: 6:35 - loss: 0.0455 - acc: 0.982 - ETA: 6:34 - loss: 0.0455 - acc:

...

 52224/143613 [=========>....................] - ETA: 13:32 - loss: 0.0419 - acc: 0.98 - ETA: 13:31 - loss: 0.0419 - acc: 0.98 - ETA: 13:31 - loss: 0.0419 - acc: 0.98 - ETA: 13:29 - loss: 0.0419 - acc: 0.98 - ETA: 13:28 - loss: 0.0419 - acc: 0.98 - ETA: 13:27 - loss: 0.0421 - acc: 0.98 - ETA: 13:26 - loss: 0.0421 - acc: 0.98 - ETA: 13:25 - loss: 0.0421 - acc: 0.98 - ETA: 13:24 - loss: 0.0421 - acc: 0.98 - ETA: 13:23 - loss: 0.0421 - acc: 0.98 - ETA: 13:22 - loss: 0.0421 - acc: 0.98 - ETA: 13:21 - loss: 0.0420 - acc: 0.98 - ETA: 13:21 - loss: 0.0420 - acc: 0.98 - ETA: 13:21 - loss: 0.0419 - acc: 0.98 - ETA: 13:20 - loss: 0.0418 - acc: 0.98 - ETA: 13:19 - loss: 0.0420 - acc: 0.98 - ETA: 13:19 - loss: 0.0419 - acc: 0.98 - ETA: 13:17 - loss: 0.0419 - acc: 0.98 - ETA: 13:16 - loss: 0.0418 - acc: 0.98 - ETA: 13:15 - loss: 0.0417 - acc: 0.98 - ETA: 13:14 - loss: 0.0418 - acc: 0.98 - ETA: 13:14 - loss: 0.0418 - acc: 0.98 - ETA: 13:13 - loss: 0.0418 - acc: 0.98 - ETA: 13:12 - loss: 0.0418 - acc

 78336/143613 [===============>..............] - ETA: 10:21 - loss: 0.0422 - acc: 0.98 - ETA: 10:20 - loss: 0.0422 - acc: 0.98 - ETA: 10:19 - loss: 0.0422 - acc: 0.98 - ETA: 10:18 - loss: 0.0422 - acc: 0.98 - ETA: 10:17 - loss: 0.0422 - acc: 0.98 - ETA: 10:16 - loss: 0.0423 - acc: 0.98 - ETA: 10:15 - loss: 0.0423 - acc: 0.98 - ETA: 10:14 - loss: 0.0423 - acc: 0.98 - ETA: 10:13 - loss: 0.0422 - acc: 0.98 - ETA: 10:13 - loss: 0.0422 - acc: 0.98 - ETA: 10:12 - loss: 0.0422 - acc: 0.98 - ETA: 10:11 - loss: 0.0422 - acc: 0.98 - ETA: 10:10 - loss: 0.0422 - acc: 0.98 - ETA: 10:09 - loss: 0.0422 - acc: 0.98 - ETA: 10:09 - loss: 0.0422 - acc: 0.98 - ETA: 10:08 - loss: 0.0421 - acc: 0.98 - ETA: 10:07 - loss: 0.0422 - acc: 0.98 - ETA: 10:06 - loss: 0.0422 - acc: 0.98 - ETA: 10:06 - loss: 0.0422 - acc: 0.98 - ETA: 10:05 - loss: 0.0422 - acc: 0.98 - ETA: 10:04 - loss: 0.0422 - acc: 0.98 - ETA: 10:03 - loss: 0.0422 - acc: 0.98 - ETA: 10:02 - loss: 0.0422 - acc: 0.98 - ETA: 10:01 - loss: 0.0422 - acc

104448/143613 [====================>.........] - ETA: 7:17 - loss: 0.0428 - acc: 0.983 - ETA: 7:16 - loss: 0.0428 - acc: 0.983 - ETA: 7:15 - loss: 0.0428 - acc: 0.983 - ETA: 7:15 - loss: 0.0428 - acc: 0.983 - ETA: 7:14 - loss: 0.0428 - acc: 0.983 - ETA: 7:13 - loss: 0.0428 - acc: 0.983 - ETA: 7:12 - loss: 0.0428 - acc: 0.983 - ETA: 7:11 - loss: 0.0428 - acc: 0.983 - ETA: 7:10 - loss: 0.0428 - acc: 0.983 - ETA: 7:09 - loss: 0.0428 - acc: 0.983 - ETA: 7:08 - loss: 0.0428 - acc: 0.983 - ETA: 7:07 - loss: 0.0428 - acc: 0.983 - ETA: 7:06 - loss: 0.0428 - acc: 0.983 - ETA: 7:05 - loss: 0.0428 - acc: 0.983 - ETA: 7:05 - loss: 0.0428 - acc: 0.983 - ETA: 7:04 - loss: 0.0428 - acc: 0.983 - ETA: 7:03 - loss: 0.0428 - acc: 0.983 - ETA: 7:02 - loss: 0.0428 - acc: 0.983 - ETA: 7:01 - loss: 0.0428 - acc: 0.983 - ETA: 7:00 - loss: 0.0428 - acc: 0.983 - ETA: 6:59 - loss: 0.0428 - acc: 0.983 - ETA: 6:58 - loss: 0.0428 - acc: 0.983 - ETA: 6:57 - loss: 0.0428 - acc: 0.983 - ETA: 6:56 - loss: 0.0428 - acc:

130560/143613 [==========================>...] - ETA: 4:17 - loss: 0.0427 - acc: 0.983 - ETA: 4:16 - loss: 0.0427 - acc: 0.983 - ETA: 4:15 - loss: 0.0427 - acc: 0.983 - ETA: 4:15 - loss: 0.0428 - acc: 0.983 - ETA: 4:14 - loss: 0.0428 - acc: 0.983 - ETA: 4:13 - loss: 0.0427 - acc: 0.983 - ETA: 4:12 - loss: 0.0428 - acc: 0.983 - ETA: 4:11 - loss: 0.0427 - acc: 0.983 - ETA: 4:10 - loss: 0.0427 - acc: 0.983 - ETA: 4:09 - loss: 0.0428 - acc: 0.983 - ETA: 4:08 - loss: 0.0427 - acc: 0.983 - ETA: 4:07 - loss: 0.0427 - acc: 0.983 - ETA: 4:07 - loss: 0.0427 - acc: 0.983 - ETA: 4:06 - loss: 0.0427 - acc: 0.983 - ETA: 4:05 - loss: 0.0427 - acc: 0.983 - ETA: 4:04 - loss: 0.0427 - acc: 0.983 - ETA: 4:03 - loss: 0.0427 - acc: 0.983 - ETA: 4:02 - loss: 0.0427 - acc: 0.983 - ETA: 4:01 - loss: 0.0427 - acc: 0.983 - ETA: 4:01 - loss: 0.0427 - acc: 0.983 - ETA: 4:00 - loss: 0.0427 - acc: 0.983 - ETA: 3:59 - loss: 0.0427 - acc: 0.983 - ETA: 3:58 - loss: 0.0427 - acc: 0.983 - ETA: 3:57 - loss: 0.0427 - acc:

143488/143613 [============================>.] - ETA: 1:24 - loss: 0.0423 - acc: 0.983 - ETA: 1:23 - loss: 0.0423 - acc: 0.983 - ETA: 1:22 - loss: 0.0424 - acc: 0.983 - ETA: 1:21 - loss: 0.0424 - acc: 0.983 - ETA: 1:20 - loss: 0.0424 - acc: 0.983 - ETA: 1:20 - loss: 0.0424 - acc: 0.983 - ETA: 1:19 - loss: 0.0424 - acc: 0.983 - ETA: 1:18 - loss: 0.0424 - acc: 0.983 - ETA: 1:17 - loss: 0.0424 - acc: 0.983 - ETA: 1:16 - loss: 0.0424 - acc: 0.983 - ETA: 1:15 - loss: 0.0424 - acc: 0.983 - ETA: 1:15 - loss: 0.0424 - acc: 0.983 - ETA: 1:14 - loss: 0.0424 - acc: 0.983 - ETA: 1:13 - loss: 0.0424 - acc: 0.983 - ETA: 1:12 - loss: 0.0424 - acc: 0.983 - ETA: 1:11 - loss: 0.0425 - acc: 0.983 - ETA: 1:10 - loss: 0.0425 - acc: 0.983 - ETA: 1:10 - loss: 0.0425 - acc: 0.983 - ETA: 1:09 - loss: 0.0425 - acc: 0.983 - ETA: 1:08 - loss: 0.0425 - acc: 0.983 - ETA: 1:07 - loss: 0.0425 - acc: 0.983 - ETA: 1:06 - loss: 0.0425 - acc: 0.983 - ETA: 1:05 - loss: 0.0425 - acc: 0.983 - ETA: 1:05 - loss: 0.0425 - acc:

In [21]:
model.load_weights(file_model)
print('Predicting....')
y_pred = model.predict(X_test_pad,batch_size=1024)
submission = pd.read_csv(file_submission_example)
submission[labels] = y_pred
submission.to_csv(file_submission, index=False)
print('Done')

Predicting....
Done


### Error Analysis

In [22]:
# this is the original data
path_train_origin = join(path_tmp, 'data', 'train.csv')
path_test_origin = join(path_tmp, 'data', 'test.csv')
X_train_origin, X_test_origin, y_train, merge_origin = GetData(path_train_origin, path_test_origin)

In [23]:
X = X_train_pad
y = y_train

y_pred = model.predict(X, batch_size=1024)
target_class = 0 # toxic
target_result = y_pred[:, target_class]
target_gt = y[labels[target_class]]


In [24]:
def Cost(fpr, tpr):
    a = 1-tpr
    b = fpr
    cost = a + 30*b
    return cost

# get the threshold that maximize the F1 score
def GetBestThreshold(gt, pred):

    fpr, tpr, thresholds = metrics.roc_curve(gt, pred)
    cost = np.inf
    threshold = None
    for idx in range(len(thresholds)):
        current_cost = Cost(fpr[idx], tpr[idx])
        if current_cost < cost:
            cost = current_cost
            threshold = thresholds[idx]
    return threshold, cost

threshold, _ = GetBestThreshold(target_gt, target_result)
print(threshold)

0.83609277


In [25]:
print(metrics.roc_auc_score(target_gt, target_result))
target_pred = target_result>threshold
mask = target_gt!=target_pred
error = np.sum(mask)
total = len(mask)
print(error)
print(total)
print((total-error)/total)

0.9849464767492032
5957
159571
0.9626686553321092


In [26]:
#X_train, X_test, y_train, merge = GetData(file_train, file_test)
misclassified_gt = target_gt[mask]
misclassified_x =  X_train[mask]
#X_train[:5]
Error0 = misclassified_x[misclassified_gt==1] # misclassify toxic as healthy
Error1 = misclassified_x[misclassified_gt==0] # misclassify healthy as toxic

In [27]:
for x in range(20):
    current = Error1.index[x]
    print(Error1[current])
    print('@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@')
    print(X_train_origin[current])
    print('######################################')

a bisexual like a homosexual a heterosexual defined sexual activity much like a year old boy attracted a girl sexually never sex still straight a person actually sexually attracted aroused sex well opposite sex bisexual
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
A Bisexual, like a homosexual or a heterosexual, is not defined by sexual activity. (Much like a 15 year old boy who is attracted to a girl sexually but has never had sex is still straight). A person who is actually sexually attracted/aroused by the same sex as well as the opposite sex is bisexual.
######################################
know sex foetus
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
"know the sex of the foetus"""
######################################
reply loser un defines vietnam part southeast asia far i know vietnam part asean used part french indochina laos shit countries anyway culture always influenced sea han chinese proper yangtze han chinese fringe indigenous tribes guangzhou guangxi admit vietnamese a bunch wanna

######################################
death utter twat said died june 3rd
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Death?!?!

What utter twat said that she died on June 3rd 2008?!?!?!
######################################
know come across a giant prick
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
you 

Do you know you come across as a giant prick?
######################################
know threating craxy doning nothing please stop threating freedom a black man beat first thats called self defence accused vilonce a threat mad note simpley telling please repeat please stop threating homie im mad kyltym rude aggenst blacks hispanic note
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
just so you know 

you are threating me and me and Craxy were doning nothing please stop threating and we have the freedom if a black man was to beat you up because you did first thats called self defence and should not be accused of vilonce this is not a threat or mad note this is simpley telling you to please repeat PLE